In [22]:
import csv
import jaconv
import re

def preprocessing(csv_path: str):
    """
    与えられたCSVファイルを読み込んだのち、以下の処理をしてから、{CSVファイル名}_preprocessed.csvとして保存する
    CSVファイルのフォーマットは"TEXT,LABEL"の2列である。

    TEXTの変換ルールは次の通り。
    1. 文字列が半角・全角スペース・改行を含む場合、その文字列を複数の文字列に分割する
    2. 記号（！,？,!,?,・,.,…,',",♪,♫）と全ての絵文字を削除する
    3. ()または（）で囲まれた文字列を削除する
    4. 半角カタカナを全角カタカナに、~を～に、-をーに変換する
    5. 2つ以上連続する～～を～に、ーーをーに変換する
    6. 空文字列を削除する

    保存する前にフィルタリングを行う。
    1. TEXTが空文字列の行を削除する
    2. TEXTとLABELの組み合わせが重複している行を削除する
    """
    # Read the CSV file
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        data = list(reader)
    
    preprocessed_data = []

    # Preprocess the TEXT column
    for i in range(len(data)):
        text, label = data[i]
        # Split the text into multiple strings if it contains spaces or newlines
        text = re.split(r'\s+', text)
        # Remove symbols
        text = [re.sub(r'[！？!?・.…\'"’”\♪♫]', '', word) for word in text]
        # Remove strings enclosed in parentheses
        text = [re.sub(r'\(.*?\)|（.*?）', '', word) for word in text]
        # Convert half-width katakana to full-width katakana
        text = [jaconv.h2z(word) for word in text]
        # Convert ~ to ～ and - to ー
        # Note: 〜(U+301C) is a different character from ～(U+FF5E
        text = [re.sub(r'[~〜]', '～', word) for word in text]
        text = [re.sub(r'-', 'ー', word) for word in text]
        # Convert multiple consecutive ～ to ～ and ーー to ー
        text = [re.sub(r'～+', '～', word) for word in text]
        text = [re.sub(r'ー+', 'ー', word) for word in text]
        
        [preprocessed_data.append([word, label]) for word in text if word != '' ]

    # Remove duplicate rows based on TEXT and LABEL combination
    preprocessed_data = [list(x) for x in set(tuple(x) for x in preprocessed_data)]

    # Sort the data by LABEL, TEXT
    preprocessed_data.sort(key=lambda x: (x[1], x[0]))

    # Save the preprocessed data to a new CSV file
    preprocessed_csv_path = csv_path.replace('.csv', '_preprocessed.csv')
    with open(preprocessed_csv_path, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(preprocessed_data)


In [23]:
preprocessing('data/train.csv')